## 1. Setup

### 1.1 Import libraries

In [6]:
from qiskit import QuantumCircuit
from qiskit import transpile
from qiskit_aer import AerSimulator
import random

### 1.2 Quantum Circuit Helper methods 

In [7]:
def run_circuit(circuit, shots=1):
    simulator = AerSimulator()
    circ = transpile(qc, simulator)

    job = simulator.run(circ, shots=shots)
    return job.result(), circ

## 2. Test Single Qubit

### 2.1 Create Encoded Qubits

In [8]:
def init_qubit(qc, idx, state):
    if state == 0:
        pass
    elif state == 1:
        qc.x(idx)
    elif state == 2:
        qc.h(idx)
    elif state == 3:
        qc.x(idx)
        qc.h(idx)
    return qc

def measure_in_basis(qc, idx, basis):
    if basis == 0:
        pass
    elif basis == 1:
        qc.h(idx)
    return qc

# |0>, |+> => 0
# |1>, |-> => 1
def state_to_bit(state):
    if state % 2 == 0:
        return 0
    else:
        return 1

def validate(state, guess, result):
    if guess == 0 and state in [0, 1]:
        guess_correct = True
    elif guess == 1 and state in [2, 3]:
        guess_correct = True
    else:
        guess_correct = False
        
    result_correct = state_to_bit(state) == result

    return guess_correct, result_correct


### 2.2 Run 2048 times

After 2048 samples, the number of correctly measured qubits should be ~75%. There is a 50% chance of guessing the correct basis.

In [ ]:
n = 2048
correct_guesses = 0
correct_results = 0


for i in range(n):
    state = random.randint(0, 3)
    qc = QuantumCircuit(1,1)
    qc = init_qubit(qc, 0, state)

    # guess state to measure
    # 0: standard, 1: horizontal
    guess = random.randint(0, 1)
    qc = measure_in_basis(qc, 0, guess)

    qc.measure_all(add_bits=False)

    result, circ = run_circuit(qc)
    counts = result.get_counts(circ)
    guess_correct, result_correct = validate(state, guess, int(list(counts.keys())[0]))
    
    if guess_correct:
        correct_guesses += 1
    if result_correct:
        correct_results += 1

print(f"Correct guesses: {correct_guesses} ({correct_guesses/n*100}%)")
print(f"Correct results: {correct_results} ({correct_results/n*100}%)")


## 3. Scale Number of Qubits

### 3.1 Helper methods


In [10]:
def generate_state_set(n):
    states = []
    for i in range(n):
        state = random.randint(0, 3)
        states.append(state)
    return states

def generate_circuit_from_state_set(states):
    n = len(states)
    qc = QuantumCircuit(n, n)
    for i in range(n):
        qc = init_qubit(qc, i, states[i])

    return qc

def generate_guess_set(n):
    guesses = []
    for i in range(n):
        guess = random.randint(0, 1)
        guesses.append(guess)
    return guesses

def measure_circuit(qc, guesses):
    n = len(guesses)
    for i in range(n):
        qc = measure_in_basis(qc, i, guesses[i])
    return qc


def validate_sets(states, guesses, results):
    correct_results = False
    correct_bits = 0
    for i in range(len(states)):
        state = states[i]
        state_bit = state_to_bit(state)
        result_bit = results[i]
        if state_bit == result_bit:
            correct_bits += 1

    if correct_bits == len(states):
        correct_results = True

    # confirm that the guess was correct
    correct_guesses = 0
    for i in range(len(states)):
        state = states[i]
        guess = guesses[i]
        if guess == 0 and state in [0, 1]:
            correct_guesses += 1
        elif guess == 1 and state in [2, 3]:
            correct_guesses += 1
    # number of correct guesses, correct result(true/false), number of correct bits
    return correct_guesses, correct_results, correct_bits

    

def highest_probability_result(counts):
    # choose the key with the highest value
    return max(counts, key=counts.get)
    



### 3.2 Test up to 20 Qubits

In [ ]:
measurement_results = []
# run the experiment for 1 to 20 qubits
for i in range(20):
    total_correct_guesses = 0
    total_correct_results = 0
    total_correct_bits = 0
    # run the experiment n times
    # with unique sets of qubits and guesses
    for j in range(n):
        # generate a set of n encoded qubits
        state = generate_state_set(i + 1)
        qc = generate_circuit_from_state_set(state)

        # generate a set of n basis guesses
        guess = generate_guess_set(i + 1)
        # measure each qubit in the basis
        qc = measure_circuit(qc, guess)

        # run the circuit
        qc.measure_all(add_bits=False)
        result, circ = run_circuit(qc)
        counts = result.get_counts(circ)
        results_str = highest_probability_result(counts)
        results = [int(c) for c in results_str]
        
        correct_guesses, correct_results, correct_bits = validate_sets(state, guess, results)
        total_correct_guesses += correct_guesses
        total_correct_bits += correct_bits
        if correct_results:
            total_correct_results += 1
    print(f"{i + 1} Qubits")
    print(f"Correct guesses: {total_correct_guesses} ({total_correct_guesses/((i + 1) * n)*100}%)")
    print(f"Correct bits: {total_correct_bits} ({total_correct_bits/((i + 1) * n)*100}%)")
    print(f"Correct results: {total_correct_results} ({total_correct_results/n*100}%)")
    measurement_results.append({
        "qubits": i + 1,
        "correct_guesses": total_correct_guesses,

        "correct_bits": total_correct_bits,
        "correct_results": total_correct_results,
        "correct_results_percent": total_correct_results/n*100
    })

### 3.3 Plot Results

In [29]:
import matplotlib.pyplot as plt

correct_results = [r["correct_results_percent"] for r in measurement_results]
correct_guesses = []
for i in range(20):
    correct_guesses.append(measurement_results[i]["correct_guesses"] / ((i + 1) * n) * 100)
correct_bits = []
for i in range(20):
    correct_bits.append(measurement_results[i]["correct_bits"] / ((i + 1) * n) * 100)
    
qubits = [r["qubits"] for r in measurement_results]

### 3.3.1 Plot Actual Measurement vs Theoretical Measurement

In [ ]:
plt.plot(qubits, [(3/4)**n * 100 for n in qubits], label="Theoretical chance of measuring all n qubits correctly - (3/4)^n")
plt.plot(qubits, correct_results, label="Actual % of correctly measuring all n qubits")

plt.legend()

plt.xlabel("Number of Qubits")
plt.ylabel("Percentage")
plt.title("Actual vs Theoretical Measurements of Guessing Qubit States")

plt.xticks(range(1, 21))
plt.yticks(range(0, 101, 10))
plt.show()



Wiesner's paper states that there's a 75% chance of measuring the correct result from a single encoded qubit. This scales expontentially as (3/4)^n where n is the number of qubits in the system. (blue curve)

In this graph, we can see that our actual measurements (orange) follows the curve, but not too tightly.

### 3.3.2 Plot Guessing individual bit results

In [ ]:
plt.plot(qubits, correct_guesses, label="% of correct guesses")
plt.plot(qubits, correct_bits, label="% of correct bits")
plt.legend()
plt.xlabel("Number of Qubits")
plt.ylabel("Percentage")
plt.title("Correct Guesses and Correct Bits")
plt.xticks(range(1, 21))
plt.yticks(range(50, 90, 10))
plt.show()

There is only two basis (standard and diagonal), therefore it's there's only a 50% chance of guessing the correct basis. (blue curve)

For a single qubit system, the chance of yielding the correct bit at random is 75%. However, this seems to converge towards 50% as the number of qubits increase. (orange curve)

## 4 Transmitting a Hashed message 

### 4.1 Import libraries

In [48]:
import hashlib

### 4.2 Create a SHA256 Hash

In [52]:
secret_message = "https://youtu.be/dQw4w9WgXcQ"

m = hashlib.sha256()
m.update(secret_message.encode("utf-8"))
hashed_message = m.hexdigest()

### 4.3 Convert Hash into Binary Data

In [ ]:
# convert each hexadecimal character to binary bytes
binary_messages = []
for c in hashed_message:
    binary_messages.append(format(int(c, 16), "04b"))

print(hashed_message)
print(binary_messages)

### 4.4 Encode into Qubits

In [ ]:
circuits = []
states = []
for binary in binary_messages:
    qc = QuantumCircuit(4, 4)
    state_cache = []
    for i in range(4):
        basis = random.randint(0, 1)
        bit = int(binary[i])
        state = 2 * basis + bit
        state_cache.append(state)
        qc = init_qubit(qc, i, state)
    circuits.append(qc)
    states.append(state_cache)

print(states)

### 4.5 Measure Received Qubits by selecting Random Basis

In [ ]:
constructed_hash = ""

for i in range(len(circuits)):
    qc = circuits[i]
    chosen_basis = [random.randint(0, 1) for i in range(4)]
    qc = measure_circuit(qc, chosen_basis)    
    qc.measure_all(add_bits=False)
    
    result, circ = run_circuit(qc)
    counts = result.get_counts(circ)
    results_str = highest_probability_result(counts)
    # convert the binary string to a hexadecimal character
    constructed_hash += hex(int(results_str, 2))[2:]

print(constructed_hash)
